In [3]:
import numpy as np
from tqdm import tqdm
from PIL import Image

import torch.nn as nn
from torch.autograd import Variable
import torch

import torchvision.transforms as transforms
import torchvision.models as models
import torchvision

In [77]:
transform = transforms.Compose(
            [
            # 左右対称の画像を生成してデータ量を増やす(Augmentation)
            # transforms.RandomHorizontalFlip(),

            # 256*256に画像サイズを変更
            transforms.Resize((256, 256)),

            # PyTorchで利用するTensorの形式にデータを変換
             transforms.ToTensor(),

            ])

train_data = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
val_data = torchvision.datasets.ImageFolder(root='./data/val', transform=transform)
test_data = torchvision.datasets.ImageFolder(root='./data/test', transform=transform)

data_size  = {"train":len(train_data), "validation":len(val_data)}

train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False)

dataloaders  = {"train":train_loader, "validation":val_loader}

In [51]:
model = models.resnet18(pretrained=True)
print(model)

# ネットワークの重みを固定
for parameter in model.parameters():
    parameter.requires_grad = False

# 最終的な出力を2個に設定
model.fc = nn.Linear(512, 2)

if torch.cuda.is_available():
    model.cuda()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

/home/student/anaconda3/envs/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/student/anaconda3/envs/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [74]:
lr = 1e-4
epoch = 1
optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss().cuda()

def train_model(model, criterion, optimizer, num_epochs):
    for epoch in tqdm(range(num_epochs)):
        epoch_loss = 0
        epoch_acc = 0

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            current_loss = 0.0
            current_corrects = 0

            for data in dataloaders[phase]:
                inputs, labels = data

                inputs = inputs.cuda()
                labels = labels.cuda()

                outputs = model(inputs)

                _, preds = torch.max(outputs.data, 1)

                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                current_loss += loss.item() * inputs.size(0) 
                current_corrects += torch.sum(preds == labels)

            epoch_loss = current_loss / data_size[phase]

            epoch_acc = current_corrects.item() / data_size[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    return model

trained_model = train_model(model, criterion, optim, epoch)

  0%|          | 0/1 [00:00<?, ?it/s]

train Loss: 0.2408 Acc: 0.9080


100%|██████████| 1/1 [00:17<00:00, 17.28s/it]

validation Loss: 0.2731 Acc: 0.8860


In [53]:
torch.save(trained_model.state_dict(), './MRI-model.pth')

In [70]:
model.load_state_dict(torch.load('./MRI-model.pth'))

<All keys matched successfully>

In [78]:
def test_model(model, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    current_loss = 0.0
    current_corrects = 0

    for data in test_loader:
        inputs, labels = data

        inputs = inputs.cuda()
        labels = labels.cuda()

        outputs = model(inputs)

        _, preds = torch.max(outputs.data, 1)

        loss = criterion(outputs, labels)

        current_loss += loss.item() * inputs.size(0) 
        current_corrects += torch.sum(preds == labels)

    epoch_loss = current_loss / len(test_data)

    epoch_acc = current_corrects.item() / len(test_data)

    print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    return model

test_model = test_model(model, criterion)

Loss: 0.2695 Acc: 0.8954
